In [61]:
import os
import numpy as np
import pandas as pd

dataPath = (os.getcwd().replace("\\", "/")) + "/../자료/"
nonSharedDataPath = os.getcwd().replace("\\", "/") + "/../../공유못함/"

In [62]:
pTDF = pd.read_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', sep=',', encoding='euc-kr')
pTDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27560766 entries, 0 to 27560765
Data columns (total 6 columns):
 #   Column     Dtype
---  ------     -----
 0   기준_날짜      int64
 1   시작_행정동_ID  int64
 2   종료_행정동_ID  int64
 3   승객수        int64
 4   시작_행정구_ID  int64
 5   종료_행정구_ID  int64
dtypes: int64(6)
memory usage: 1.2 GB


In [63]:
mappingDF = pd.read_csv(dataPath + '행정동매핑.csv', encoding='euc-kr')
mappingDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   행정구코드   426 non-null    int64 
 1   시군구     426 non-null    object
 2   행정동코드   426 non-null    int64 
 3   읍면동     426 non-null    object
dtypes: int64(2), object(2)
memory usage: 13.4+ KB


In [64]:
c1 = (pTDF['기준_날짜'] == 0)
c2 = (pTDF['시작_행정동_ID'] == 0)
c3 = (pTDF['종료_행정동_ID'] == 0)
(c1 | c2 | c3).unique()
pTDF = pTDF[(c1 | c2 | c3) == False]

In [65]:
def getGuID(x):
    x[4] = int(x[1] / 100)
    x[5] = int(x[2] / 100)
    
    return x

checkDF = pTDF[:5]
checkDF['시작_행정구_ID'] = ""
checkDF['종료_행정구_ID'] = ""

checkDF = pd.DataFrame(np.apply_along_axis(getGuID, 1, checkDF), columns=checkDF.columns.to_list())
checkDF
# checkDF.apply(getGuID, axis=1)

C:\Users\bigdata\AppData\Local\Temp\ipykernel_11704\855479904.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkDF['시작_행정구_ID'] = ""
C:\Users\bigdata\AppData\Local\Temp\ipykernel_11704\855479904.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkDF['종료_행정구_ID'] = ""


,기준_날짜,시작_행정동_ID,종료_행정동_ID,승객수,시작_행정구_ID,종료_행정구_ID
0,20220101,1112056,2305057,6,11120,23050
1,20220101,1113071,1113070,46,11130,11130
2,20220101,1102071,1125074,8,11020,11250
3,20220101,1105053,1110065,149,11050,11100
4,20220101,1104052,1102065,43,11040,11020


In [66]:
pTDF['시작_행정구_ID'] = ""
pTDF['종료_행정구_ID'] = ""

def getGuID(x):
    x[4] = int(x[1] / 100)
    x[5] = int(x[2] / 100)
    
    return x

pTDF2 = pd.DataFrame(np.apply_along_axis(getGuID, 1, pTDF), columns=pTDF.columns.to_list())
#pTDF.apply(getGuID, axis=1)
pTDF2

,기준_날짜,시작_행정동_ID,종료_행정동_ID,승객수,시작_행정구_ID,종료_행정구_ID
0,20220101,1112056,2305057,6,11120,23050
1,20220101,1113071,1113070,46,11130,11130
2,20220101,1102071,1125074,8,11020,11250
3,20220101,1105053,1110065,149,11050,11100
4,20220101,1104052,1102065,43,11040,11020
...,...,...,...,...,...,...
27560761,20220430,3102154,1104066,8,31021,11040
27560762,20220430,1112074,1115054,1,11120,11150
27560763,20220430,2308060,1123066,3,23080,11230
27560764,20220430,1123052,1123052,966,11230,11230


In [ ]:
pTDF2.to_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', encoding='euc-kr', index=False)

In [ ]:
# 자치구별 대중교통현황 자료 만들기

pTDF2 = pd.read_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', encoding='euc-kr')
pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])] = pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])].astype('str')
pTDF2.info()

In [ ]:
shGroup = pTDF2.groupby(pTDF2.columns[4:].append(pTDF2.columns[0:1]).to_list(), as_index=False).agg('sum').sort_values('승객수', ascending=False)
shGroup.info()

In [ ]:
shGroupStart = shGroup.groupby(['시작_행정구_ID', '기준_날짜'], as_index=False).agg('sum')
shGroupStart.rename({'시작_행정구_ID' : '행정구'}, inplace=True, axis=1)
shGroupStart

In [ ]:
shGroupEnd = shGroup.groupby(['종료_행정구_ID', '기준_날짜'], as_index=False).agg('sum')
shGroupEnd.rename({'종료_행정구_ID' : '행정구'}, inplace=True, axis=1)
shGroupEnd

In [ ]:
sumDF = shGroupStart.merge(shGroupEnd, on=['행정구', '기준_날짜'])
sumDF['승객수'] = ((sumDF['승객수_x'] + sumDF['승객수_y']) / 2).astype('int')
sumDF['행정구'] = sumDF['행정구'].astype('int')
sumDF = sumDF[['행정구', '승객수', '기준_날짜']][sumDF['행정구'] < 20000]

In [ ]:
print(sumDF.head(1))
print(mappingDF.head(1))

In [ ]:
sumDF['시군구'] = ""

def mappingGu(x):
    for i in mappingDF.index:
        if (x[0] == mappingDF.iloc[i, 0]):
            x[3] = mappingDF.iloc[i, 1]
            break;
    return x
            
sumDF = pd.DataFrame(np.apply_along_axis(mappingGu, 1, sumDF), columns=sumDF.columns)
sumDF

In [ ]:
sumDF.to_csv(dataPath + '대중교통자치구별현황.csv', encoding='euc-kr', index=False)

In [67]:
#행정동별 대중교통현황 자료 만들기

pTDF2 = pd.read_csv(nonSharedDataPath + '행정동별대중교통수단통합.csv', encoding='euc-kr')
pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])] = pTDF2[pTDF2.columns[:3].append(pTDF2.columns[4:])].astype('str')
pTDF2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27560766 entries, 0 to 27560765
Data columns (total 6 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   기준_날짜      object
 1   시작_행정동_ID  object
 2   종료_행정동_ID  object
 3   승객수        int64 
 4   시작_행정구_ID  object
 5   종료_행정구_ID  object
dtypes: int64(1), object(5)
memory usage: 1.2+ GB


In [70]:
shGroupStart = pTDF2.groupby(['시작_행정동_ID', '기준_날짜', '시작_행정구_ID'], as_index=False).agg('sum')
shGroupStart.rename({'시작_행정동_ID' : '행정동', '시작_행정구_ID' : '시군구'}, inplace=True, axis=1)
shGroupStart

,행정동,기준_날짜,시군구,승객수
0,1101053,20220101,11010,20161
1,1101053,20220102,11010,21299
2,1101053,20220103,11010,53671
3,1101053,20220104,11010,56770
4,1101053,20220105,11010,57669
...,...,...,...,...
84425,4413358,20220426,44133,95
84426,4413358,20220427,44133,1023
84427,4413358,20220428,44133,1097
84428,4413358,20220429,44133,1090


In [71]:
shGroupEnd = pTDF2.groupby(['종료_행정동_ID', '기준_날짜', '종료_행정구_ID'], as_index=False).agg('sum')
shGroupEnd.rename({'종료_행정동_ID' : '행정동', '종료_행정구_ID' : '시군구'}, inplace=True, axis=1)
shGroupEnd

,행정동,기준_날짜,시군구,승객수
0,1101053,20220101,11010,20888
1,1101053,20220102,11010,22103
2,1101053,20220103,11010,54991
3,1101053,20220104,11010,58251
4,1101053,20220105,11010,59025
...,...,...,...,...
84403,4413358,20220426,44133,57
84404,4413358,20220427,44133,929
84405,4413358,20220428,44133,991
84406,4413358,20220429,44133,887


In [88]:
sumDF = shGroupStart.merge(shGroupEnd, on=['행정동', '기준_날짜', '시군구'])
sumDF['승객수'] = ((sumDF['승객수_x'] + sumDF['승객수_y']) / 2).astype('int')
sumDF['행정동'] = sumDF['행정동'].astype('int')
sumDF['시군구'] = sumDF['시군구'].astype('int')
sumDF = sumDF[['시군구', '행정동', '승객수', '기준_날짜']][sumDF['시군구'] < 20000]

In [89]:
print(sumDF.head(1))
print(mappingDF.head(1))

     시군구      행정동    승객수     기준_날짜
0  11010  1101053  20524  20220101
   행정구코드  시군구    행정동코드  읍면동
0  11010  종로구  1101053  사직동


In [90]:
sumDF['시군구명칭'] = ""
sumDF['행정동명칭'] = ""

def mappingGu(x):
    for i in mappingDF.index:
        if (x[0] == mappingDF.iloc[i, 0]):
            x[4] = mappingDF.iloc[i, 1]
            x[5] = mappingDF.iloc[i, 3]
            break;
    return x
            
sumDF = pd.DataFrame(np.apply_along_axis(mappingGu, 1, sumDF), columns=sumDF.columns)
sumDF

,시군구,행정동,승객수,기준_날짜,시군구명칭,행정동명칭
0,11010,1101053,20524,20220101,종로구,사직동
1,11010,1101053,21701,20220102,종로구,사직동
2,11010,1101053,54331,20220103,종로구,사직동
3,11010,1101053,57510,20220104,종로구,사직동
4,11010,1101053,58347,20220105,종로구,사직동
...,...,...,...,...,...,...
47058,11250,1125074,1183,20220426,강동구,강일동
47059,11250,1125074,16513,20220427,강동구,강일동
47060,11250,1125074,16654,20220428,강동구,강일동
47061,11250,1125074,17058,20220429,강동구,강일동


In [91]:
sumDF.to_csv(dataPath + '대중교통행정동별현황.csv', encoding='euc-kr', index=False)

In [95]:
#행정동별 교통량현황 자료 만들기

carMovingDF = pd.read_csv(dataPath + '행정동추가.csv', sep=',', encoding='euc-kr')
carMovingSumDF = pd.read_csv(dataPath + '통행합계.csv', sep=',', encoding='euc-kr')
carMovingSumDF

,일자,요일,지점명,지점번호,방향,통행량
0,20220101,일,성산로(금화터널),A-01,유입,0
1,20220102,일,성산로(금화터널),A-01,유입,0
2,20220103,월,성산로(금화터널),A-01,유입,0
3,20220104,화,성산로(금화터널),A-01,유입,0
4,20220105,수,성산로(금화터널),A-01,유입,0
...,...,...,...,...,...,...
33355,20220426,화,신월여의지하도로,F-10,유출,47150
33356,20220427,수,신월여의지하도로,F-10,유출,47572
33357,20220428,목,신월여의지하도로,F-10,유출,49836
33358,20220429,금,신월여의지하도로,F-10,유출,49690


In [96]:
def Sung

,지점번호,지점명칭,검지기 유형,위도,경도,주소,도로명 주소,유입 방향,유출방향,행정동
0,A-01,성산로(금화터널),지자기,37.568588,126.948436,서울시 서대문구 신촌동 1-142,0,[성산로]봉원고가차도->독립문역,[성산로]독립문역->봉원고가차도,신촌동
1,A-02,사직로(사직터널),지자기,37.572298,126.962853,서울시 종로구 행촌동 1-186,0,[사직로]독립문역->사직단,[사직로]사직단->독립문역,행촌동
2,A-03,자하문로(자하문터널),지자기,37.588831,126.968548,서울시 종로구 청운동 24-6,0,[자하문로]석파정->청운초등학교,[자하문로]청운초등학교->석파정,청운동
3,A-04,대사관로(삼청터널),지자기,37.596359,126.984209,서울시 성북구 성북동 산25-148,0,[삼청로]삼청각->삼청공원입구,[삼청로]삼청공원입구->삼청각,성북동
4,A-05,율곡로(안국역),지자기,37.576000,126.984342,서울시 종로구 경운동 90-4,서울시 종로구 율곡로 46,[율곡로]안국역->안국동사거리,[율곡로]안국동사거리->안국역,경운동
...,...,...,...,...,...,...,...,...,...,...
134,F-06,경부고속도로,영상,37.493135,127.022533,서울시 서초구 서초동 1748-26,0,[경부고속도로]서초2교 -> 서초IC,[경부고속도로]서초IC -> 서초2교,서초동
135,F-07,분당수서로,영상,37.497648,127.087195,서울시 강남구 일원동 467-1,0,[분당수서로]수서IC->탄천1교,[분당수서로]탄천1교->수서IC,일원동
136,F-08,강남순환로(관악터널),지자기,37.449096,126.926169,서울시 금천구 시흥동 산 93-1,0,[강남순환로]금천요금소->관악IC,[노들로]관악IC->금천요금소,시흥동
137,F-09,서부간선지하도로,루프,37.468942,126.883673,서울시 금천구 가산동 338-10,서울시 금천구 가산디지털2로 5,[서부간선지하도로]금천IC->성산대교남단,[서부간선지하도로]성산대교남단->금천IC,가산동
